In [26]:
using Revise

using MFAScan.EnsembleTest
using MFA.Ensemble
using Random
using LatticeModels
using Plots
using Lattices
using LinearAlgebra
using ProfileView
using KrylovKit
plotly()

Plots.PlotlyBackend()

In [27]:
struct CubicParams
    L::Int
    W::Float64
end

In [28]:
function f(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = cubic(L = p.L, M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L*L) .- 0.5)
end

function g(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = square(M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L) .- 0.5)
end

function h(p::CubicParams; rng = Random.GLOBAL_RNG) #Aubry Andre
    H = chain(N = p.L) .+ p.W*Diagonal(cos.(2π*(1:L)*(1 + √5)/2 .+ 2*pi*rand(rng)))
end

h (generic function with 1 method)

In [29]:
L = 18; W = 3.5; E_c = 0.0; E_del = 0.5
l = [1]
q = collect(0:0.3:3)

p = CubicParams(L, W)
ltc = Lattice3D(L, L, L, 1)
p_MFA = MFAParameters(ltc, l, q)
prepare_MFA!(p_MFA)

In [30]:
@code_warntype scan_ταf(f, p, rng = MersenneTwister(1234), E_c, E_del, p_MFA, R=10, nev = 10)

MethodInstance for (::MFAScan.EnsembleTest.var"#scan_ταf##kw")(::NamedTuple{(:rng, :R, :nev), Tuple{MersenneTwister, Int64, Int64}}, ::typeof(scan_ταf), ::typeof(f), ::CubicParams, ::Float64, ::Float64, ::MFAParameters{Lattice3D{Int64}})
  from (::MFAScan.EnsembleTest.var"#scan_ταf##kw")(::Any, ::typeof(scan_ταf), f::Function, params, E_c::Float64, E_del::Float64, p_MFA::MFAParameters) in MFAScan.EnsembleTest at /Users/kimyeongjun/codes/project/julia/module/MFAScan/src/ensemble_test.jl:55
Arguments
  _::Core.Const(MFAScan.EnsembleTest.var"#scan_ταf##kw"())
  @_2::NamedTuple{(:rng, :R, :nev), Tuple{MersenneTwister, Int64, Int64}}
  @_3::Core.Const(MFAScan.EnsembleTest.scan_ταf)
  f::Core.Const(f)
  params::CubicParams
  E_c::Float64
  E_del::Float64
  p_MFA::MFAParameters{Lattice3D{Int64}}
Locals
  c::Float64
  nev::Int64
  rng::MersenneTwister
  isherm::Bool
  noaverage::Bool
  kwargs_eig...::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}
  R::Int64
  @_16::Float64
  @_1

In [31]:
@profview E, gipr, μqlnμ, τ, α, f_α = scan_ταf(f, p, rng = MersenneTwister(1234), E_c, E_del, p_MFA, R=10, nev = 10)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [32]:
@profview E, gipr, μqlnμ, τ, α, f_α = mt_scan_ταf(f, p, rng = MersenneTwister(1234), E_c, E_del, p_MFA, R=10, nev = 10)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [33]:
function ldiv2!(y, F, x)
    y .= F\x 
end

H = f(p)
x = rand(size(H, 1))
y = rand(size(H, 1))

@time begin 
    F = lu(H)
    ldiv!(y, F, x);
end


@time begin 
    F = factorize(H)
    ldiv2!(y, F, x);
end



  0.204067 seconds (233.69 k allocations: 70.680 MiB, 1.57% gc time, 42.49% compilation time)
  0.252776 seconds (10.66 k allocations: 36.331 MiB, 3.58% compilation time)


5832-element Vector{Float64}:
   0.4092736078657222
  -6.567574739724421
   5.330376598271519
  -4.695589565091767
 -14.948054908928428
  -3.7501122834070277
   2.97210845115751
 -12.77065399066046
   4.333905855921993
  13.729775056544913
  14.012257923786198
   1.718202604355477
   3.846603640696923
   ⋮
   3.8599079065234996
  20.926211813200098
   0.6582419988975658
 -19.276594925352587
   0.37831243072227094
   0.6850999489139582
   1.4967493886453354
 -20.192096518274955
   1.1842198731275875
   2.278908373223473
  -4.049062581918807
   6.477601517590785